---
jupyter: python3
toc: true
date: 2021-11-17
categories: [Python, Linux]
author: limyj0708
title: 리눅스 Shell 명령어 Python 스크립트에서 실행하기 + Crontab
comments:
  giscus: 
    repo: quarto-dev/quarto-docs
---

# 리눅스 Shell 명령어 Python 스크립트에서 실행하기 + Crontab
> "그리고 Crontab으로도 실행해보기"

## 1. 배경
 - 회사에 업무를 위한 소형 개인 서버로 쓰는 NUC가 있다.
   - ssh로 연결하여 사용
 - 사내 와이파이에 연결되어 있는데, 아주 가끔씩 할당된 IP가 바뀐다.
 - 이럴 때마다 모니터를 연결해서 ifconfig로 ip 주소를 확인할 수는 없는 노릇이다.
 - 일주일에 한 번씩, 서버가 나에게 현재 자신의 ip가 뭔지 보내줬으면 좋겠다.

## 2. 실행

### 1. Shell 명령어

```sh
ifconfig [원하는 네트워크 인터페이스명] | grep -Eo '([0-9]{1,3}[\.]){3}[0-9]{1,3}'
```
- grep
  - -E : 표현을 확장 정규 표현식으로 해석
  - -o : 매칭되는 문자열만 표시
  

### 2. Python 스크립트에서

```Python
import subprocess
import requests

regex_ipv4 = '([0-9]{1,3}[\.]){3}[0-9]{1,3}' #ipv4를 추출하는 정규식
ps = subprocess.Popen(("ifconfig", "원하는 네트워크 인터페이스명"), stdout=subprocess.PIPE)
output = subprocess.check_output(("grep", "-Eo", regex_ipv4), stdin=ps.stdout)
ps.wait()
ipv4_internal = str(output).split('\\n')[0][2:]
# 사내에서 접근 가능한 IP주소만 추출함

TARGET_URL = 'https://notify-api.line.me/api/notify'
TOKEN = '라인 Notify에서 발급받은 토큰 입력'
# 요청합니다.
response = requests.post(
    TARGET_URL,
    headers={
    'Authorization': 'Bearer ' + TOKEN
    },
    data={
    'message': f'NUC IP : {ipv4_internal}'
    }
)
```

 - Shell에서처럼 Pipe(|)를 쓸 수 없다.
   - Popen에 shell=True를 넘겨주면 되긴 하는데, 일반적으로 shell에서 명령을 내리는 것 처럼 별도의 유효성 검사 없이 실행이 되기 때문에 shell injection에 취약하게 된다.
 - 그래서 쪼개서 실행시켜야 한다. Popen으로 ifconfig를 실행시키고, 그 출력값을 check_output에 연결하여 최종 출력값을 만든다.
 - 추출한 IP를 Line Notify를 통해 라인으로 받는다.
 
 
### 3. Crontab에서
 
```sh
 PATH=/usr/bin:/usr/sbin:/sbin:/usr/local/bin
 # ifconfig, grep을 잘 실행시키기 위한 환경 지정
 
 # ssh 접속용 사내 Wifi ipv4 전송용
 # 매주 월요일 오전 10시에 전송 
00 10 * * 1 /usr/bin/python3.9 /home/limyj0708/Code/ipv4_internal_alarm/ipv4_internal_alarm.py >> /home/limyj0708/Code/ipv4_internal_alarm/cron_log.log 2>&1
 ```

### 4. 결과

 - 지정한 Line Notify 봇을 통해 IP가 잘 날아온다.

## Reference

1. [Shell=True는 Shell Injection에 취약](https://hashcode.co.kr/questions/2567/%ED%8C%8C%EC%9D%B4%EC%8D%AC-shelltrue-%EC%9D%B4-%EB%AC%B4%EC%97%87%EC%9D%84-%ED%95%98%EB%8A%94%EC%A7%80-%EC%9E%98-%EB%AA%A8%EB%A5%B4%EA%B2%A0%EC%8A%B5%EB%8B%88%EB%8B%A4)
2. [Popen 클래스 개괄](http://theyoonicon.com/python-popen-%ED%81%B4%EB%9E%98%EC%8A%A4/)
3. [Subprocess 모듈 사용법](https://soooprmx.com/python-subprocess-1/)